In [1]:
!pip -q install cadquery trimesh plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.6/182.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.3/740.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.8/539.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.1/245.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.3 MB/s eta 0:00:00


In [2]:
import cadquery as cq
from cadquery import exporters

cube = cq.Workplane("XY").box(200, 200, 200)  # 20cm = 200mm
exporters.export(cube, "cube_200mm.stl")

print("Wrote cube_200mm.stl")

Wrote cube_200mm.stl


In [5]:
import trimesh
import numpy as np
import plotly.graph_objects as go

m = trimesh.load("cube_200mm.stl")

# --- エッジ（線分）抽出 ---
edges = m.edges_unique  # (E,2) vertex index
V = m.vertices

# Plotlyの線は、(x,y,z)を区切りでつなぐ
xs, ys, zs = [], [], []
for a, b in edges:
    xs += [V[a,0], V[b,0], None]
    ys += [V[a,1], V[b,1], None]
    zs += [V[a,2], V[b,2], None]

# --- 面（Mesh） ---
mesh_trace = go.Mesh3d(
    x=V[:,0], y=V[:,1], z=V[:,2],
    i=m.faces[:,0], j=m.faces[:,1], k=m.faces[:,2],
    opacity=0.9
)

# --- 線（Edges） ---
edge_trace = go.Scatter3d(
    x=xs, y=ys, z=zs,
    mode="lines",
    line=dict(width=3),  # 太さ指定
    name="edges"
)

fig = go.Figure(data=[mesh_trace, edge_trace])
fig.update_layout(
    scene_aspectmode="data",
    margin=dict(l=0, r=0, t=0, b=0)
)
fig.show()

In [6]:
import cadquery as cq
from cadquery import exporters
from math import sqrt

# 寸法（mm）
side = 200.0
height = 200.0

tri_h = side * sqrt(3) / 2.0

p1 = (-side/2.0, -tri_h/3.0)
p2 = ( side/2.0, -tri_h/3.0)
p3 = (0.0,       2.0*tri_h/3.0)

base = cq.Workplane("XY").polyline([p1, p2, p3]).close()
apex = cq.Workplane("XY").workplane(offset=height).circle(1.0)  # 1.0mmで安定する

pyramid_solid = cq.Solid.makeLoft([base.val(), apex.val()], ruled=False)

pyramid = cq.Workplane("XY").newObject([pyramid_solid])

exporters.export(pyramid, "tri_pyramid_200_200.stl")
print("Wrote tri_pyramid_200_200.stl")

Wrote tri_pyramid_200_200.stl


In [7]:
import trimesh
import plotly.graph_objects as go

m = trimesh.load("tri_pyramid_200_200.stl")

V = m.vertices
edges = m.edges_unique

xs, ys, zs = [], [], []
for a, b in edges:
    xs += [V[a,0], V[b,0], None]
    ys += [V[a,1], V[b,1], None]
    zs += [V[a,2], V[b,2], None]

mesh_trace = go.Mesh3d(
    x=V[:,0], y=V[:,1], z=V[:,2],
    i=m.faces[:,0], j=m.faces[:,1], k=m.faces[:,2],
    opacity=0.9
)

edge_trace = go.Scatter3d(
    x=xs, y=ys, z=zs,
    mode="lines",
    line=dict(width=3),
    name="edges"
)

fig = go.Figure(data=[mesh_trace, edge_trace])
fig.update_layout(
    scene_aspectmode="data",
    margin=dict(l=0, r=0, t=0, b=0)
)
fig.show()